In [9]:
a = [0,3,45,5,3,2,8,5,0,4,2,4,5,7,6,8,0]

In [3]:
a.remove(0)

In [4]:
a

[3, 45, 5, 3, 2, 8, 5, 0, 4, 2, 4, 5, 7, 6, 8, 0]

In [10]:
list(set(a)-set([0]))

[2, 3, 4, 5, 6, 7, 8, 45]

In [2]:
import torch
from scripts import mnist, train_utils, architectures, train

In [3]:
layers = [
    {"n_in": 784, "n_out": 16, "batchnorm": False},
    {"n_out": 32, "batchnorm": True},
    {"n_out": 64, "batchnorm": True},
    {"n_out": 10, "batchnorm": True}
]
net = architectures.MLPCustom(layers)
print(net)

MLPCustom(
  (layers): Sequential(
    (0): Flatten()
    (1): Linear(in_features=784, out_features=16, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=16, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=32, out_features=64, bias=True)
    (8): ReLU()
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=64, out_features=10, bias=True)
    (11): ReLU()
  )
)


In [4]:
[pars.abs().flatten() for pars in net.parameters()]

[tensor([0.0205, 0.0292, 0.0177,  ..., 0.0224, 0.0023, 0.0195],
        grad_fn=<ViewBackward>),
 tensor([0.0192, 0.0108, 0.0196, 0.0104, 0.0230, 0.0265, 0.0009, 0.0174, 0.0185,
         0.0105, 0.0260, 0.0160, 0.0315, 0.0027, 0.0118, 0.0313],
        grad_fn=<AbsBackward>),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        grad_fn=<AbsBackward>),
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        grad_fn=<AbsBackward>),
 tensor([0.1449, 0.1347, 0.1354, 0.1508, 0.1591, 0.0585, 0.2240, 0.1679, 0.2359,
         0.2481, 0.1333, 0.1536, 0.2199, 0.2184, 0.0987, 0.2391, 0.2218, 0.0429,
         0.0913, 0.2087, 0.1382, 0.2180, 0.2392, 0.2365, 0.0301, 0.0269, 0.1886,
         0.1191, 0.1891, 0.0598, 0.1176, 0.1218, 0.0680, 0.1679, 0.0834, 0.0975,
         0.0752, 0.1322, 0.1437, 0.1893, 0.0864, 0.1981, 0.2277, 0.1117, 0.0417,
         0.2261, 0.1613, 0.0468, 0.1224, 0.2328, 0.0944, 0.2382, 0.1320, 0.1615,
         0.0749, 0.1045, 0

In [12]:
[pars.shape for pars in net.parameters()]

[torch.Size([16, 784]),
 torch.Size([16]),
 torch.Size([16]),
 torch.Size([16]),
 torch.Size([32, 16]),
 torch.Size([32]),
 torch.Size([32]),
 torch.Size([32]),
 torch.Size([64, 32]),
 torch.Size([64]),
 torch.Size([64]),
 torch.Size([64]),
 torch.Size([10, 64]),
 torch.Size([10])]

In [52]:
fl1 = torch.cat([pars.abs().flatten() for pars in net.parameters()], dim=0)
fl1, fl1.shape

(tensor([0.0205, 0.0292, 0.0177,  ..., 0.0663, 0.0378, 0.0821],
        grad_fn=<CatBackward>),
 torch.Size([16090]))

In [54]:
fl1 = fl1[fl1.nonzero()].reshape(fl1[fl1.nonzero()].shape[0])
fl1, fl1.shape

(tensor([0.0205, 0.0292, 0.0177,  ..., 0.0663, 0.0378, 0.0821],
        grad_fn=<ViewBackward>),
 torch.Size([12872]))

In [49]:
fl1 = fl1.reshape(fl1.shape[0])
fl1, fl1.shape

(tensor([0.0205, 0.0292, 0.0177,  ..., 0.0663, 0.0378, 0.0821],
        grad_fn=<ViewBackward>),
 torch.Size([12872]))

In [42]:
fl2 = torch.unique(torch.cat([pars.abs().flatten() for pars in net.parameters()], dim=0).sort()[0])
fl2, fl2.shape

(tensor([0.0000, 0.0085, 0.0085,  ..., 0.2490, 0.2492, 1.0000],
        grad_fn=<NotImplemented>),
 torch.Size([12752]))

In [8]:
flat = torch.cat([pars.abs().flatten() for pars in net.parameters()], dim=0)
flat.shape

torch.Size([16090])

In [14]:
flat = torch.cat([pars.abs().flatten() for pars in net.parameters()], dim=0)

flat = flat.sort()[0]
pruning_rate = 0.2
position = int(pruning_rate*flat.shape[0])
thresh = flat[position]

mask = [torch.where( pars.abs() >= thresh, torch.ones(pars.shape), torch.zeros(pars.shape) ) for pars in net.parameters()]

In [15]:
for m in mask:
    print(m)

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 0.,  ..., 0., 0., 1.],
        [1., 0., 1.,  ..., 1., 0., 1.],
        [1., 1., 0.,  ..., 1., 0., 1.]])
tensor([1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.],
        [

In [31]:
for param, m in zip(net.parameters(), mask):
        param.data *= m
        print(param)

Parameter containing:
tensor([[-0.0205,  0.0292,  0.0177,  ...,  0.0309,  0.0136,  0.0104],
        [ 0.0208,  0.0000,  0.0000,  ...,  0.0185, -0.0223,  0.0323],
        [ 0.0184,  0.0088,  0.0000,  ..., -0.0299,  0.0247,  0.0301],
        ...,
        [-0.0324,  0.0250, -0.0000,  ..., -0.0000, -0.0000, -0.0338],
        [ 0.0328,  0.0000,  0.0155,  ...,  0.0176, -0.0000,  0.0229],
        [-0.0192,  0.0222,  0.0000,  ...,  0.0224,  0.0000,  0.0195]],
       requires_grad=True)
Parameter containing:
tensor([-0.0192,  0.0108, -0.0196,  0.0104,  0.0230,  0.0265,  0.0000, -0.0174,
         0.0185,  0.0105,  0.0260,  0.0160,  0.0315,  0.0000, -0.0118, -0.0313],
       requires_grad=True)
Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)
Parameter containing:
tensor([[ 0.1449,  0.1347,  0.1354,  0.15